# CFP Propensity Version 1.0

### LIBRARY IMPORTS

In [2]:
'''Importing Required Libraries'''
import copy
from collections import OrderedDict
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier,SGDRegressor
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import make_pipeline
import joblib

### DATA IMPORT

In [3]:
'''Importing the data for modelling'''
cfp_data = pd.read_csv(r"D:\Dbeaver Out\CFP_Data_Green_202301051222.csv", na_values=['',' ','NH', 'NA', '[]'])
# cfp_data = pd.read_csv(r"D:\Dbeaver Out\CFP_Data_Red_202301051318.csv", na_values=['',' ','NH', 'NA', '[]'])
# cfp_data = pd.read_csv(r"C:\Users\Pon kebha.DESKTOP-PI8M9QL\CFU_Data_Red_ltd2301_202301201931.csv", na_values=['',' ','NH', 'NA', '[]'])
# cfp_data = pd.read_csv(r"C:\Users\Pon kebha.DESKTOP-PI8M9QL\CFU_Data_Green_ltd2301_202301231133.csv", na_values=['',' ','NH', 'NA', '[]'])

In [4]:
cfp_data.shape

(303296, 84)

In [5]:
cfp_data.columns

Index(['user_id', 'ltd', 'customer_type', 'nsaleable', 'latest_login_date',
       'salary_account', 'cfp_subscribed', 'monthly_income', 'credit_score',
       'age', 'pincode', 'marital_status', 'employment_type',
       'first_profile_date', 'latest_profile_date', 'city_band',
       'ptp_last_three_months_flag', 'positive_response_rate',
       'contactability', 'email_login_flag', 'unique_days_logged_in_flag',
       'sms_login_flag', 'last_login_vintage', 'negative_status_flag',
       'settled_flag', 'open_total_ratio', 'totl_neg_ratio',
       'saleable_accounts_count', 'cfp_interest', 'auto_loan_count',
       'gold_loan_count', 'consumer_loan_count', 'housing_loan_count',
       'credit_card_count', 'personal_loan_count', 'two_wheeler_count',
       'open_accounts', 'closed_accounts', 'settled_accounts',
       'written_off_accounts', 'flows_accounts', 'secured', 'unsecured',
       'open_negative_status_count', 'individual_account', 'joint_account',
       'total_amt_sanction

In [6]:
cfp_data.head()

,user_id,ltd,customer_type,nsaleable,latest_login_date,salary_account,cfp_subscribed,monthly_income,credit_score,age,pincode,marital_status,employment_type,first_profile_date,latest_profile_date,city_band,ptp_last_three_months_flag,positive_response_rate,contactability,email_login_flag,unique_days_logged_in_flag,sms_login_flag,last_login_vintage,negative_status_flag,settled_flag,open_total_ratio,totl_neg_ratio,saleable_accounts_count,cfp_interest,auto_loan_count,gold_loan_count,consumer_loan_count,housing_loan_count,credit_card_count,personal_loan_count,two_wheeler_count,open_accounts,closed_accounts,settled_accounts,written_off_accounts,flows_accounts,secured,unsecured,open_negative_status_count,individual_account,joint_account,total_amt_sanctioned,total_balance,credit_limit,repayment_tenure,total_emi_amount,hdfc_neg_flag,citi_neg_flag,sbi_neg_flag,icici_neg_flag,cc_neg_flag,pl_neg_flag,cl_neg_flag,auto_loan_close_count_last3mnths,gold_loan_close_count_last3mnths,consumer_loan_close_count_last3mnths,housing_loan_close_count_last3mnths,credit_card_close_count_last3mnths,personal_loan_close_count_last3mnths,two_wheeler_close_count_last3mnths,individual_close_account_last3mnths,closed_accounts_last3mnths,secured_close_last3mnths,unsecured_close_last3mnths,negative_status_close_count_last3mnths,joint_account_close_last3mnths,max_score,score_on_m9,score_on_m3,score_on_m12,score_on_m6,neg_acc_on_m9,neg_acc_on_m3,neg_acc_on_m12,neg_acc_on_m6,max_dpd_on_m9,max_dpd_on_m3,max_dpd_on_m12,max_dpd_on_m6
0,2042889,2210,Green,0,NaN,HDFC,0,33000.00,750,48.00,600099.00,Married,Salaried,2017-06-19 08:53:55.000,2021-02-13 18:21:43.000,Chennai,NaN,0.00,0.00,Zero,MoreThanThree,Zero,BeyondM12,MoreThanThree,1,0.39,0.00,0,NaN,0.00,0.00,2.00,1.00,6.00,8.00,0.00,7.00,11.00,11.00,0.00,7.00,1.00,17.00,0.00,18.00,0.00,146.00,66.00,28.00,21.00,20053.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,775.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26573549,2210,Green,0,2022-09-14 12:35:11.000,NaN,0,20000.00,700,22.00,457001.00,Single,Salaried,2022-09-14 12:36:14.000,2022-09-14 12:36:14.000,Missing/Others,NaN,0.00,0.00,Zero,One,Zero,M1toM2,MoreThanThree,1,0.67,0.00,0,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,2.00,1.00,1.00,0.00,2.00,2.00,1.00,0.00,1.00,2.00,42.00,25.00,NaN,11.00,14229.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,700.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23202365,2211,Green,0,2022-10-17 05:07:33.000,NaN,0,50217.50,750,31.00,636705.00,Married,NaN,2022-01-23 18:56:09.000,2022-10-17 05:07:40.000,Missing/Others,NaN,0.00,0.00,Zero,Two,Zero,M1toM2,MoreThanThree,1,0.21,0.00,0,0.00,0.00,10.00,0.00,0.00,0.00,43.00,0.00,12.00,45.00,45.00,0.00,12.00,10.00,47.00,0.00,57.00,0.00,189.00,24.00,NaN,42.00,154211.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,0.00,3.00,3.00,0.00,3.00,0.00,0.00,750.00,750.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.00,NaN,NaN,NaN
3,21726339,2211,Green,0,2022-10-18 17:08:22.000,NaN,0,19078.84,725,42.00,395007.00,Married,Salaried,2021-06-03 15:08:57.000,2022-10-10 14:09:19.000,Missing/Others,NaN,0.00,0.00,Zero,MoreThanThree,MoreThanOnce,M1toM2,MoreThanThree,1,0.63,0.00,0,0.00,0.00,0.00,14.00,0.00,0.00,0.00,0.00,10.00,6.00,6.00,0.00,10.00,0.00,16.00,0.00,16.00,0.00,76.00,25.00,NaN,18.00,16764.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,775.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26883702,2210,Green,0,2022-10-01 18:35:34.000,NaN,0,17289.90,650,52.00,422103.00,Married,NaN,2022-09-26 11:24:18.000,2022-09-26 11:24:18.000,Missing/Others,NaN,0.00,0.00,Zero,Two,One,M1toM2,MoreThanThree,1,0.38,0.00,0,0.00,0.00,0.00,5.00,0.00,0.00,0.00,0.00,3.00,5.00,5.00,0.00,2.00,0.00,8.00,0.00,8.00,0.00,19.00,3.00,NaN,3.00,1334.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,0.00,0.00,650.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

## PRE-PROCESSING

#### Handling Null Values

In [7]:
cfp_data.isnull().sum()

user_id                                        0
ltd                                            0
customer_type                                  0
nsaleable                                      0
latest_login_date                          93979
salary_account                            273862
cfp_subscribed                                 0
monthly_income                             20320
credit_score                                   0
age                                           26
pincode                                       29
marital_status                             19076
employment_type                            89072
first_profile_date                             0
latest_profile_date                            0
city_band                                      0
ptp_last_three_months_flag                273416
positive_response_rate                         0
contactability                                 0
email_login_flag                               0
unique_days_logged_i

In [8]:
cfp_data.describe(percentiles=[0.05,0.10,0.25,0.5,0.75,0.9,0.95,0.99]).transpose()

,count,mean,std,min,5%,10%,25%,50%,75%,90%,95%,99%,max
user_id,303296.00,14741316.48,10477774.08,71.00,876380.50,1559472.50,3642662.75,16355112.00,25584449.75,26977172.50,27358960.50,27863021.10,28023083.00
ltd,303296.00,2211.00,0.82,2210.00,2210.00,2210.00,2210.00,2211.00,2212.00,2212.00,2212.00,2212.00,2212.00
nsaleable,303296.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
cfp_subscribed,303296.00,0.01,0.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00
monthly_income,282976.00,92056.23,13277136.24,-4583.33,0.00,4166.67,14473.10,19569.30,25672.08,41666.67,60000.00,100000.00,5000000000.00
credit_score,303296.00,727.14,48.26,425.00,625.00,675.00,700.00,725.00,750.00,775.00,775.00,800.00,900.00
age,303270.00,33.97,15.17,-5965.00,22.00,23.00,27.00,32.00,39.00,47.00,53.00,64.00,2011.00
pincode,303267.00,436002.77,199043.56,1118.00,110081.00,131039.00,282001.00,431001.00,562114.00,700135.00,784125.00,845437.00,999999.00
ptp_last_three_months_flag,29880.00,0.07,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00
positive_response_rate,303296.00,0.03,0.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00


In [9]:
''' Replacing with 0s and Harcoded Values'''
cfp_data.ptp_last_three_months_flag.fillna(0, inplace= True)
cfp_data.totl_neg_ratio.fillna(0, inplace= True)
cfp_data.max_score.fillna(cfp_data.max_score.mean(),inplace=True)
cfp_data.auto_loan_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.gold_loan_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.consumer_loan_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.housing_loan_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.credit_card_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.personal_loan_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.two_wheeler_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.individual_close_account_last3mnths.fillna(0, inplace= True)
cfp_data.secured_close_last3mnths.fillna(0, inplace= True)
cfp_data.closed_accounts_last3mnths.fillna(0, inplace= True)
cfp_data.negative_status_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.joint_account_close_last3mnths.fillna(0, inplace= True)
cfp_data.unsecured_close_last3mnths.fillna(0, inplace= True)

cfp_data.auto_loan_count.fillna(0, inplace= True)
cfp_data.gold_loan_count.fillna(0, inplace= True)
cfp_data.consumer_loan_count.fillna(0, inplace= True)
cfp_data.housing_loan_count.fillna(0, inplace= True)
cfp_data.credit_card_count.fillna(0, inplace= True)
cfp_data.personal_loan_count.fillna(0, inplace= True)
cfp_data.two_wheeler_count.fillna(0, inplace= True)
cfp_data.individual_account.fillna(0, inplace= True)
cfp_data.open_accounts.fillna(0, inplace= True)
cfp_data.settled_accounts.fillna(0, inplace= True)
cfp_data.secured.fillna(0, inplace= True)
cfp_data.unsecured.fillna(0, inplace= True)
cfp_data.open_negative_status_count.fillna(0, inplace= True)
cfp_data.joint_account.fillna(0, inplace= True)
cfp_data.total_balance.fillna(0, inplace= True)
cfp_data.written_off_accounts.fillna(0, inplace= True)
cfp_data.flows_accounts.fillna(0, inplace= True)
cfp_data.open_accounts.fillna(0, inplace= True)
cfp_data.closed_accounts.fillna(0, inplace= True)


cfp_data.hdfc_neg_flag.fillna(0, inplace= True)
cfp_data.citi_neg_flag.fillna(0, inplace= True)
cfp_data.sbi_neg_flag.fillna(0, inplace= True)        
cfp_data.icici_neg_flag.fillna(0, inplace= True)         
cfp_data.cc_neg_flag.fillna(0, inplace= True)       
cfp_data.pl_neg_flag.fillna(0, inplace= True)
cfp_data.cl_neg_flag.fillna(0, inplace= True)

cfp_data.total_balance.fillna(cfp_data.total_balance.mean(),inplace=True)
cfp_data.total_emi_amount.fillna(cfp_data.total_emi_amount.mean(),inplace=True)
cfp_data.total_amt_sanctioned.fillna(cfp_data.total_amt_sanctioned.mean(),inplace=True)

cfp_data.score_on_m9.fillna(0, inplace= True)
cfp_data.score_on_m3.fillna(0, inplace= True)
cfp_data.score_on_m12.fillna(0, inplace= True)
cfp_data.score_on_m6.fillna(0, inplace= True)
cfp_data.neg_acc_on_m9.fillna(0, inplace= True)
cfp_data.neg_acc_on_m3.fillna(0, inplace= True)
cfp_data.neg_acc_on_m12.fillna(0, inplace= True)
cfp_data.neg_acc_on_m6.fillna(0, inplace= True)
cfp_data.max_dpd_on_m9.fillna(0, inplace= True)
cfp_data.max_dpd_on_m3.fillna(0, inplace= True)
cfp_data.max_dpd_on_m12.fillna(0, inplace= True)
cfp_data.max_dpd_on_m6.fillna(0, inplace= True)

# cfp_data.score_track.fillna(0, inplace= True)
# cfp_data.neg_acc_track.fillna(0, inplace= True)
# cfp_data.max_dpd_track.fillna(0, inplace= True)

cfp_data.cfp_interest.fillna(0, inplace= True)

cfp_data.salary_account.fillna('Unknown', inplace=True)

In [10]:
'''Replacing based on median'''
median_marriage_age=np.median(cfp_data.age[cfp_data['marital_status'] == 'Married'])
def marital_na_filler(marital_status, age): 
    try: 
        if np.isnan(marital_status): 
            if age >= median_marriage_age: 
                return 'Married' 
            else: 
                return 'Single' 
        else: 
            return (marital_status) 
    except: 
        return (marital_status)
cfp_data.marital_status = cfp_data.apply(lambda x: marital_na_filler(x.marital_status, x.age), axis = 1)

In [11]:
'''Replacing based on mean'''
cfp_data.age.fillna(np.mean(cfp_data.age), inplace= True)
cfp_data.monthly_income.fillna(np.mean(cfp_data.monthly_income), inplace= True)

In [12]:
'''Replacing based on Mode'''
mode_credit_limit=cfp_data.credit_limit.mode()
cfp_data.credit_limit.fillna(int(mode_credit_limit),inplace=True)

mode_repayment_tenure=cfp_data.repayment_tenure.mode()
cfp_data.repayment_tenure.fillna(int(mode_repayment_tenure),inplace=True)

In [13]:
''' Replacing FLAG in city_band with 'Missing/Others' '''
cfp_data.city_band = cfp_data.city_band.apply(lambda x: 'Missing/Others' if x == 'FLAG' else x)

In [14]:
'''Replacing Salary Account Bank with Categories'''
def getFormattedSalaryAccount(Bank):
    try:
        formatted_employment_type = ""
        SBI = ['SBI GROUP','SBI']
        AXIS = ['AXIS','Axis']
        HDFC = ['HDFC','HDFC Bank']
        KOTAK = ['kotak mahindra bank','KOTAK','kotak']
        BOI = ['BANK OF INDIA']
        PNB = ['PUNJAB NATIONAL', 'PNB']
        if Bank in SBI:
            return "SBI"
        elif Bank in AXIS:
            return "AXIS"
        elif Bank in HDFC:
            return "HDFC"
        elif Bank in KOTAK:
            return "KOTAK"
        elif Bank in PNB:
            return "PNB"
        elif Bank in BOI:
            return "BOI"
        elif Bank == "YESBANK":
            return "YESBANK"
        elif Bank == "ANDHRA":
            return "ANDHRA"
        elif Bank == "IDBI":
            return "IDBI"
        elif Bank == "CANARA":
            return "CANARA"
        elif Bank == "UNION":
            return "UNION"
        elif Bank == "ICICI":
            return "ICICI"
        elif Bank == "receiveByCashOrCheque":
            return "I receive by cash or cheque"
        elif Bank == "CITI":
            return "CITI"
        elif Bank == 'I receive by cash':
            return "I receive by cash"
        elif Bank == 'I receive by cheque':
            return "I receive by cheque"
        else:
            return 'Other'
    except:
        return employment_type
cfp_data.salary_account = cfp_data.salary_account.apply(lambda x: getFormattedSalaryAccount(x))
cfp_data.salary_account.value_counts()

Other                          284475
SBI                              6214
HDFC                             5011
ICICI                            3703
AXIS                             2497
CITI                              491
I receive by cash                 358
YESBANK                           336
I receive by cheque               133
KOTAK                              41
IDBI                               13
BOI                                11
PNB                                 9
I receive by cash or cheque         2
ANDHRA                              1
UNION                               1
Name: salary_account, dtype: int64

In [15]:
'''Format Employment and handle null values'''
def getFormattedEmploymentType(employment_type):
    try:
        formatted_employment_type = ""
        salaried_list = ['Salaried','salaried','Salaried Doctor','Working Executive','Student','Salaried doctor', 'salaried', 'salariedDoctor']
        self_employed_list = ['Self Employed', 'Self employed', 'Self employed professional', 'Self-Employed', 'selfemployee', 'selfEmployedProfessional','Self employed business', 'selfEmployedBusiness', 'Self Employed Business']
        if employment_type in salaried_list:
            return "Salaried"
        elif employment_type in self_employed_list:
            return "Self Employed"
        else:
            return 'Salaried'
    except:
        return employment_type
cfp_data.employment_type = cfp_data.employment_type.apply(lambda x: getFormattedEmploymentType(x))

In [16]:
''' Income and Age to positive if negative '''

cfp_data.monthly_income = cfp_data.monthly_income.apply(lambda x: (x * -1) if x < 0 else x)
cfp_data.age = cfp_data.age.apply(lambda x: (x * -1) if x < 0 else x)

In [17]:
cfp_data.isnull().sum()
# cfp_data.salary_account.value_counts()

user_id                                       0
ltd                                           0
customer_type                                 0
nsaleable                                     0
latest_login_date                         93979
salary_account                                0
cfp_subscribed                                0
monthly_income                                0
credit_score                                  0
age                                           0
pincode                                      29
marital_status                                0
employment_type                               0
first_profile_date                            0
latest_profile_date                           0
city_band                                     0
ptp_last_three_months_flag                    0
positive_response_rate                        0
contactability                                0
email_login_flag                              0
unique_days_logged_in_flag              

In [18]:
cfp_backup=copy.deepcopy(cfp_data)
cfp_data.drop(['latest_login_date','ltd','first_profile_date','latest_profile_date','pincode'],axis=1,inplace=True) #'ltd',

#### Probe and Handle Outliers

In [19]:
numerical_columns=list(cfp_data.select_dtypes(include='number').columns)
exclude=['user_id','monthly_income', 'age','cfp_interest','positive_response_rate', 'contactability',
         'ptp_last_three_months_flag','cfp_subscribed'] #,'cfp_subscribed'
for feature in exclude:
    numerical_columns.remove(feature)
print(numerical_columns)

['nsaleable', 'credit_score', 'settled_flag', 'open_total_ratio', 'totl_neg_ratio', 'saleable_accounts_count', 'auto_loan_count', 'gold_loan_count', 'consumer_loan_count', 'housing_loan_count', 'credit_card_count', 'personal_loan_count', 'two_wheeler_count', 'open_accounts', 'closed_accounts', 'settled_accounts', 'written_off_accounts', 'flows_accounts', 'secured', 'unsecured', 'open_negative_status_count', 'individual_account', 'joint_account', 'total_amt_sanctioned', 'total_balance', 'credit_limit', 'repayment_tenure', 'total_emi_amount', 'hdfc_neg_flag', 'citi_neg_flag', 'sbi_neg_flag', 'icici_neg_flag', 'cc_neg_flag', 'pl_neg_flag', 'cl_neg_flag', 'auto_loan_close_count_last3mnths', 'gold_loan_close_count_last3mnths', 'consumer_loan_close_count_last3mnths', 'housing_loan_close_count_last3mnths', 'credit_card_close_count_last3mnths', 'personal_loan_close_count_last3mnths', 'two_wheeler_close_count_last3mnths', 'individual_close_account_last3mnths', 'closed_accounts_last3mnths', 'sec

In [ ]:
sns.displot(cfp_data[numerical_columns[20]])

In [20]:
'''Percentile values'''
Q05=cfp_data.quantile(0.05)
Q10=cfp_data.quantile(0.10)
Q95=cfp_data.quantile(0.95)

In [21]:
'''Capping Upper Limit'''
cfp_data['monthly_income'] = np.where(cfp_data['monthly_income']>Q95.monthly_income,Q95.monthly_income,cfp_data['monthly_income'])
cfp_data['nsaleable'] = np.where(cfp_data['nsaleable']>Q95.nsaleable,Q95.nsaleable,cfp_data['nsaleable'])
for feature in numerical_columns:
    cfp_data[feature] = np.where(cfp_data[feature]>Q95[feature],Q95[feature],cfp_data[feature])


'''Capping Upper and Lower Limit'''
cfp_data['age'] = np.where(cfp_data['age']>Q95.age,Q95.age,np.where(cfp_data['age']<Q05.age,Q05.age,cfp_data['age']))

In [22]:
'''Finding Skewness'''
def find_skew(data,numerical_columns):
    skew_list={}
    for feature in numerical_columns:
        try:
            skew_value=(3*(data[feature].mean()-data[feature].median()))/data[feature].std()
            if ((round(skew_value,2)<-3) or (round(skew_value,2)>3)):
                skew_list[feature]='Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww'
            else:
                skew_list[feature]=round(skew_value,2)
        except ZeroDivisionError:
            skew_list[feature]='Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww'
    return skew_list
find_skew(cfp_data,numerical_columns)

{'nsaleable': 'Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww',
 'credit_score': 0.08,
 'settled_flag': -1.36,
 'open_total_ratio': 0.11,
 'totl_neg_ratio': 'Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww',
 'saleable_accounts_count': 'Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww',
 'auto_loan_count': 1.35,
 'gold_loan_count': 1.2,
 'consumer_loan_count': 1.14,
 'housing_loan_count': 1.19,
 'credit_card_count': 0.89,
 'personal_loan_count': 1.34,
 'two_wheeler_count': 'Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww',
 'open_accounts': 1.07,
 'closed_accounts': 1.31,
 'settled_accounts': 1.29,
 'written_off_accounts': 'Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww',
 'flows_accounts': 0.84,
 'secured': 0.77,
 'unsecured': 0.93,
 'open_negative_status_count': 'Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww',
 'individual_account': 0.95,
 'joint_account': 1.4,
 'total_amt_sanctioned': 0.86,
 'tota

In [ ]:
#Finding better transformation 
#def test_transformers(data,columns):
#     count=len(columns)
#     pt = PowerTransformer()
#     qt = QuantileTransformer(n_quantiles=500, output_distribution='normal')
#     fig = plt.figure(figsize=(20,300))
#     j = 1
#     for i in columns:
#         array = np.array(data[i]).reshape(-1, 1)
#         y = pt.fit_transform(array)
#         x = qt.fit_transform(array)
#         plt.subplot(count,3,j)
#         sns.histplot(array, bins = 50, kde = True)
#         plt.title(f"Original Distribution for {i}")
#         plt.subplot(count,3,j+1)
#         sns.histplot(x, bins = 50, kde = True)
#         plt.title(f"Quantile Transform for {i}")
#         plt.subplot(count,3,j+2)
#         sns.histplot(y, bins = 50, kde = True)
#         plt.title(f"Power Transform for {i}")
#         j += 3
# test_transformers(cfp_data,numerical_columns)

In [23]:
'''Transforming features - Adjusting for skewness'''
qt = QuantileTransformer(n_quantiles=1000, output_distribution='normal')
for i in numerical_columns:
    array = np.array(cfp_data[i]).reshape(-1, 1)
    try:
        cfp_data[i] = qt.fit_transform(array)
    except:
        pass
find_skew(cfp_data,numerical_columns)

{'nsaleable': 3.0,
 'credit_score': 1.44,
 'settled_flag': -1.36,
 'open_total_ratio': 0.16,
 'totl_neg_ratio': 3.0,
 'saleable_accounts_count': 3.0,
 'auto_loan_count': 1.35,
 'gold_loan_count': 1.43,
 'consumer_loan_count': -1.45,
 'housing_loan_count': 1.23,
 'credit_card_count': -1.82,
 'personal_loan_count': -1.32,
 'two_wheeler_count': 3.0,
 'open_accounts': -0.14,
 'closed_accounts': -0.46,
 'settled_accounts': -0.5,
 'written_off_accounts': 3.0,
 'flows_accounts': -0.36,
 'secured': -1.81,
 'unsecured': -0.14,
 'open_negative_status_count': 3.0,
 'individual_account': 0.0,
 'joint_account': 1.45,
 'total_amt_sanctioned': 0.22,
 'total_balance': -0.45,
 'credit_limit': 0.72,
 'repayment_tenure': 0.24,
 'total_emi_amount': -1.1,
 'hdfc_neg_flag': 3.0,
 'citi_neg_flag': 3.0,
 'sbi_neg_flag': 3.0,
 'icici_neg_flag': 3.0,
 'cc_neg_flag': 3.0,
 'pl_neg_flag': 3.0,
 'cl_neg_flag': 3.0,
 'auto_loan_close_count_last3mnths': 3.0,
 'gold_loan_close_count_last3mnths': 3.0,
 'consumer_loan_

### Pickling Data

In [24]:
joblib.dump(cfp_data, 'cfp_clean_data_Green.pkl')
# joblib.dump(cfp_data, 'cfp_clean_data_Red_ltd.pkl')

['cfp_clean_data_Green.pkl']

In [25]:
cfp_clean_data=joblib.load('cfp_clean_data_Green.pkl')
# cfp_clean_data=joblib.load('cfp_clean_data_Red_ltd.pkl')

### Removing Unwanted Columns

In [26]:
user_ids=pd.concat([cfp_clean_data['user_id'].reset_index(drop = True)], axis = 1)
cfp_clean_data=cfp_clean_data.drop(['user_id'],axis=1)

In [27]:
cfp_clean_data.head()

,customer_type,nsaleable,salary_account,cfp_subscribed,monthly_income,credit_score,age,marital_status,employment_type,city_band,ptp_last_three_months_flag,positive_response_rate,contactability,email_login_flag,unique_days_logged_in_flag,sms_login_flag,last_login_vintage,negative_status_flag,settled_flag,open_total_ratio,totl_neg_ratio,saleable_accounts_count,cfp_interest,auto_loan_count,gold_loan_count,consumer_loan_count,housing_loan_count,credit_card_count,personal_loan_count,two_wheeler_count,open_accounts,closed_accounts,settled_accounts,written_off_accounts,flows_accounts,secured,unsecured,open_negative_status_count,individual_account,joint_account,total_amt_sanctioned,total_balance,credit_limit,repayment_tenure,total_emi_amount,hdfc_neg_flag,citi_neg_flag,sbi_neg_flag,icici_neg_flag,cc_neg_flag,pl_neg_flag,cl_neg_flag,auto_loan_close_count_last3mnths,gold_loan_close_count_last3mnths,consumer_loan_close_count_last3mnths,housing_loan_close_count_last3mnths,credit_card_close_count_last3mnths,personal_loan_close_count_last3mnths,two_wheeler_close_count_last3mnths,individual_close_account_last3mnths,closed_accounts_last3mnths,secured_close_last3mnths,unsecured_close_last3mnths,negative_status_close_count_last3mnths,joint_account_close_last3mnths,max_score,score_on_m9,score_on_m3,score_on_m12,score_on_m6,neg_acc_on_m9,neg_acc_on_m3,neg_acc_on_m12,neg_acc_on_m6,max_dpd_on_m9,max_dpd_on_m3,max_dpd_on_m12,max_dpd_on_m6
0,Green,-5.20,HDFC,0,33000.00,0.37,48.00,Married,Salaried,Chennai,0.00,0.00,0.00,Zero,MoreThanThree,Zero,BeyondM12,MoreThanThree,5.20,-0.23,-5.20,-5.20,0.00,-5.20,-5.20,0.41,1.24,1.33,1.10,-5.20,0.81,0.88,0.88,-5.20,0.88,0.18,1.01,-5.20,0.92,-5.20,1.23,1.29,1.10,0.83,0.49,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,0.74,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20
1,Green,-5.20,Other,0,20000.00,-0.76,22.00,Single,Salaried,Missing/Others,0.00,0.00,0.00,Zero,One,Zero,M1toM2,MoreThanThree,5.20,0.61,-5.20,-5.20,0.00,-5.20,-5.20,-5.20,1.24,0.15,-5.20,-5.20,-0.44,-0.69,-0.68,-5.20,-0.34,0.64,-1.19,-5.20,-1.37,5.20,-0.06,0.35,-0.20,0.31,0.34,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-1.11,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20
2,Green,-5.20,Other,0,50217.50,0.37,31.00,Married,Salaried,Missing/Others,0.00,0.00,0.00,Zero,Two,Zero,M1toM2,MoreThanThree,5.20,-0.83,-5.20,-5.20,0.00,-5.20,5.20,-5.20,-5.20,-5.20,5.20,-5.20,1.48,5.20,5.20,-5.20,1.53,5.20,5.20,-5.20,5.20,-5.20,1.49,0.31,-0.20,1.54,1.42,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,5.20,-5.20,5.20,5.20,-5.20,5.20,-5.20,-5.20,-0.03,5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20
3,Green,-5.20,Other,0,19078.84,-0.24,42.00,Married,Salaried,Missing/Others,0.00,0.00,0.00,Zero,MoreThanThree,MoreThanOnce,M1toM2,MoreThanThree,5.20,0.50,-5.20,-5.20,0.00,-5.20,-5.20,5.20,-5.20,-5.20,-5.20,-5.20,1.24,0.41,0.41,-5.20,1.31,-5.20,0.95,-5.20,0.81,-5.20,0.62,0.35,-0.20,0.70,0.41,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,0.74,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20
4,Green,-5.20,Other,0,17289.90,-1.45,52.00,Married,Salaried,Missing/Others,0.00,0.00,0.00,Zero,Two,One,M1toM2,MoreThanThree,5.20,-0.26,-5.20,-5.20,0.00,-5.20,-5.20,1.09,-5.20,-5.20,-5.20,-5.20,-0.09,0.28,0.28,-5.20,-0.34,-5.20,0.30,-5.20,0.16,-5.20,-0.63,-0.68,-0.20,-0.85,-0.44,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,5.20,5.20,-5.20,5.20,-5.20,-5.20,-1.77,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20,-5.20


### Segregate Columns for Encoding and Scaling

In [28]:
exemption_features=['cfp_subscribed']
id_columns=['user_id']

In [29]:
categorical_features=list(cfp_clean_data.select_dtypes(exclude='number').columns)
categorical_features

['customer_type',
 'salary_account',
 'marital_status',
 'employment_type',
 'city_band',
 'email_login_flag',
 'unique_days_logged_in_flag',
 'sms_login_flag',
 'last_login_vintage',
 'negative_status_flag']

In [30]:
numeric_features=list(cfp_clean_data.select_dtypes(include='number').columns)
numeric_features
for feature in exemption_features:
    numeric_features.remove(feature)

In [31]:
'''Encoding categorical features with dummies'''
dummies = pd.get_dummies(data = cfp_clean_data[categorical_features], drop_first=True)
dummies.head()

,salary_account_AXIS,salary_account_BOI,salary_account_CITI,salary_account_HDFC,salary_account_I receive by cash,salary_account_I receive by cash or cheque,salary_account_I receive by cheque,salary_account_ICICI,salary_account_IDBI,salary_account_KOTAK,salary_account_Other,salary_account_PNB,salary_account_SBI,salary_account_UNION,salary_account_YESBANK,marital_status_Single,employment_type_Self Employed,city_band_Chennai,city_band_Delhi/NCR,city_band_Hyderabad,city_band_Kolkata,city_band_Missing/Others,city_band_Mumbai,city_band_Pune,email_login_flag_One,email_login_flag_Zero,unique_days_logged_in_flag_One,unique_days_logged_in_flag_Three,unique_days_logged_in_flag_Two,sms_login_flag_One,sms_login_flag_Zero,last_login_vintage_M1toM2,last_login_vintage_M3toM6,last_login_vintage_M7toM12,negative_status_flag_One
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0


In [32]:
'''Scaling numeric features'''
standard_scalar = StandardScaler(with_mean=True,with_std=True)

standard_scalar.fit(cfp_clean_data[numeric_features])
numeric_data_scaled = standard_scalar.transform(cfp_clean_data[numeric_features])
numeric_data_scaled = pd.DataFrame(numeric_data_scaled,columns=numeric_features)
numeric_data_scaled.head()

,nsaleable,monthly_income,credit_score,age,ptp_last_three_months_flag,positive_response_rate,contactability,settled_flag,open_total_ratio,totl_neg_ratio,saleable_accounts_count,cfp_interest,auto_loan_count,gold_loan_count,consumer_loan_count,housing_loan_count,credit_card_count,personal_loan_count,two_wheeler_count,open_accounts,closed_accounts,settled_accounts,written_off_accounts,flows_accounts,secured,unsecured,open_negative_status_count,individual_account,joint_account,total_amt_sanctioned,total_balance,credit_limit,repayment_tenure,total_emi_amount,hdfc_neg_flag,citi_neg_flag,sbi_neg_flag,icici_neg_flag,cc_neg_flag,pl_neg_flag,cl_neg_flag,auto_loan_close_count_last3mnths,gold_loan_close_count_last3mnths,consumer_loan_close_count_last3mnths,housing_loan_close_count_last3mnths,credit_card_close_count_last3mnths,personal_loan_close_count_last3mnths,two_wheeler_close_count_last3mnths,individual_close_account_last3mnths,closed_accounts_last3mnths,secured_close_last3mnths,unsecured_close_last3mnths,negative_status_close_count_last3mnths,joint_account_close_last3mnths,max_score,score_on_m9,score_on_m3,score_on_m12,score_on_m6,neg_acc_on_m9,neg_acc_on_m3,neg_acc_on_m12,neg_acc_on_m6,max_dpd_on_m9,max_dpd_on_m3,max_dpd_on_m12,max_dpd_on_m6
0,0.00,0.23,-0.24,1.65,-0.08,-0.19,-0.20,0.45,-0.14,0.00,0.00,-0.58,-0.45,-0.48,0.60,1.75,0.96,0.81,0.00,0.47,0.54,0.54,0.00,0.50,0.60,0.55,0.00,0.50,-0.48,0.71,0.71,0.64,0.45,0.53,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.28,0.00,-0.34,-0.34,0.00,-0.32,0.00,0.00,0.38,-0.41,-0.85,-0.36,-0.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,-0.30,-0.69,-1.35,-0.08,-0.19,-0.20,0.45,0.18,0.00,0.00,-0.58,-0.45,-0.48,-1.12,1.75,0.61,-1.18,0.00,-0.12,-0.10,-0.08,0.00,-0.03,0.74,-0.61,0.00,-0.81,2.68,-0.12,0.31,-0.24,0.12,0.48,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.28,0.00,-0.34,-0.34,0.00,-0.32,0.00,0.00,-0.86,-0.41,-0.85,-0.36,-0.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.93,-0.24,-0.31,-0.08,-0.19,-0.20,0.45,-0.37,0.00,0.00,-0.58,-0.45,2.93,-1.12,-0.41,-1.00,2.10,0.00,0.78,2.28,2.28,0.00,0.79,2.14,2.74,0.00,2.94,-0.48,0.88,0.29,-0.24,0.89,0.85,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.57,0.00,2.97,2.93,0.00,3.08,0.00,0.00,-0.14,3.12,-0.85,-0.36,-0.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,-0.34,-0.48,0.96,-0.08,-0.19,-0.20,0.45,0.14,0.00,0.00,-0.58,-0.45,-0.48,2.08,-0.41,-1.00,-1.18,0.00,0.67,0.35,0.36,0.00,0.69,-1.04,0.52,0.00,0.43,-0.48,0.32,0.31,-0.24,0.36,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.28,0.00,-0.34,-0.34,0.00,-0.32,0.00,0.00,0.38,-0.41,-0.85,-0.36,-0.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,-0.41,-0.97,2.11,-0.08,-0.19,-0.20,0.45,-0.15,0.00,0.00,-0.58,-0.45,-0.48,0.81,-0.41,-1.00,-1.18,0.00,0.05,0.29,0.30,0.00,-0.03,-1.04,0.18,0.00,0.07,-0.48,-0.48,-0.13,-0.24,-0.60,0.22,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.28,0.00,2.97,2.93,0.00,3.08,0.00,0.00,-1.30,-0.41,-0.85,-0.36,-0.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [33]:
'''Concatenating all features'''
processed_dataset = pd.concat([user_ids[id_columns].reset_index(drop = True), 
                               numeric_data_scaled.reset_index(drop = True),
                               dummies.reset_index(drop = True)
                              ,cfp_clean_data[exemption_features].reset_index(drop = True)], axis=1)

processed_dataset.head()

,user_id,nsaleable,monthly_income,credit_score,age,ptp_last_three_months_flag,positive_response_rate,contactability,settled_flag,open_total_ratio,totl_neg_ratio,saleable_accounts_count,cfp_interest,auto_loan_count,gold_loan_count,consumer_loan_count,housing_loan_count,credit_card_count,personal_loan_count,two_wheeler_count,open_accounts,closed_accounts,settled_accounts,written_off_accounts,flows_accounts,secured,unsecured,open_negative_status_count,individual_account,joint_account,total_amt_sanctioned,total_balance,credit_limit,repayment_tenure,total_emi_amount,hdfc_neg_flag,citi_neg_flag,sbi_neg_flag,icici_neg_flag,cc_neg_flag,pl_neg_flag,cl_neg_flag,auto_loan_close_count_last3mnths,gold_loan_close_count_last3mnths,consumer_loan_close_count_last3mnths,housing_loan_close_count_last3mnths,credit_card_close_count_last3mnths,personal_loan_close_count_last3mnths,two_wheeler_close_count_last3mnths,individual_close_account_last3mnths,closed_accounts_last3mnths,secured_close_last3mnths,unsecured_close_last3mnths,negative_status_close_count_last3mnths,joint_account_close_last3mnths,max_score,score_on_m9,score_on_m3,score_on_m12,score_on_m6,neg_acc_on_m9,neg_acc_on_m3,neg_acc_on_m12,neg_acc_on_m6,max_dpd_on_m9,max_dpd_on_m3,max_dpd_on_m12,max_dpd_on_m6,salary_account_AXIS,salary_account_BOI,salary_account_CITI,salary_account_HDFC,salary_account_I receive by cash,salary_account_I receive by cash or cheque,salary_account_I receive by cheque,salary_account_ICICI,salary_account_IDBI,salary_account_KOTAK,salary_account_Other,salary_account_PNB,salary_account_SBI,salary_account_UNION,salary_account_YESBANK,marital_status_Single,employment_type_Self Employed,city_band_Chennai,city_band_Delhi/NCR,city_band_Hyderabad,city_band_Kolkata,city_band_Missing/Others,city_band_Mumbai,city_band_Pune,email_login_flag_One,email_login_flag_Zero,unique_days_logged_in_flag_One,unique_days_logged_in_flag_Three,unique_days_logged_in_flag_Two,sms_login_flag_One,sms_login_flag_Zero,last_login_vintage_M1toM2,last_login_vintage_M3toM6,last_login_vintage_M7toM12,negative_status_flag_One,cfp_subscribed
0,2042889,0.00,0.23,-0.24,1.65,-0.08,-0.19,-0.20,0.45,-0.14,0.00,0.00,-0.58,-0.45,-0.48,0.60,1.75,0.96,0.81,0.00,0.47,0.54,0.54,0.00,0.50,0.60,0.55,0.00,0.50,-0.48,0.71,0.71,0.64,0.45,0.53,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.28,0.00,-0.34,-0.34,0.00,-0.32,0.00,0.00,0.38,-0.41,-0.85,-0.36,-0.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,26573549,0.00,-0.30,-0.69,-1.35,-0.08,-0.19,-0.20,0.45,0.18,0.00,0.00,-0.58,-0.45,-0.48,-1.12,1.75,0.61,-1.18,0.00,-0.12,-0.10,-0.08,0.00,-0.03,0.74,-0.61,0.00,-0.81,2.68,-0.12,0.31,-0.24,0.12,0.48,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.28,0.00,-0.34,-0.34,0.00,-0.32,0.00,0.00,-0.86,-0.41,-0.85,-0.36,-0.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0
2,23202365,0.00,0.93,-0.24,-0.31,-0.08,-0.19,-0.20,0.45,-0.37,0.00,0.00,-0.58,-0.45,2.93,-1.12,-0.41,-1.00,2.10,0.00,0.78,2.28,2.28,0.00,0.79,2.14,2.74,0.00,2.94,-0.48,0.88,0.29,-0.24,0.89,0.85,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.57,0.00,2.97,2.93,0.00,3.08,0.00,0.00,-0.14,3.12,-0.85,-0.36,-0.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,1,0,0,0,0
3,21726339,0.00,-0.34,-0.48,0.96,-0.08,-0.19,-0.20,0.45,0.14,0.00,0.00,-0.58,-0.45,-0.48,2.08,-0.41,-1.00,-1.18,0.00,0.67,0.35,0.36,0.00,0.69,-1.04,0.52,0.00,0.43,-0.48,0.32,0.31,-0.24,0.36,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.28,0.00,-0.34,-0.34,0.00,-0.32,0.00,0.00,0.38,-0.41,-0.85,-0.36,-0.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0
4,26883702,0.00,-0.41,-0.97,2.11,-0.08,-0.19,-0.20,0.45,-0.15,0.00,0.00,-0.58,-0.45,-0.48,0.81,-0.41,-1.00,-1.18,0.00,0.05,0.29,0.30,0.00,-0.03,-

In [44]:
'''Eliminating worst features to increase binding'''
'''red'''
worst_features=['auto_loan_close_count_last3mnths',
 'cc_neg_flag',
 'citi_neg_flag',
 'cl_neg_flag',
 'consumer_loan_close_count_last3mnths',
 'credit_card_close_count_last3mnths',
 'gold_loan_close_count_last3mnths',
 'hdfc_neg_flag',
 'housing_loan_close_count_last3mnths',
 'icici_neg_flag',
 'joint_account_close_last3mnths',
 'max_dpd_on_m12',
 'max_dpd_on_m3',
 'max_dpd_on_m6',
 'max_dpd_on_m9',
 'neg_acc_on_m12',
 'neg_acc_on_m3',
 'neg_acc_on_m6',
 'neg_acc_on_m9',
 'negative_status_close_count_last3mnths',
 'negative_status_flag_One',
 'nsaleable',
 'open_negative_status_count',
 'pl_neg_flag',
 'salary_account_BOI',
 'salary_account_CITI',
 'salary_account_I receive by cash or cheque',
 'salary_account_I receive by cheque',
 'salary_account_IDBI',
 'salary_account_PNB',
 'salary_account_UNION',
 'salary_account_YESBANK',
 'saleable_accounts_count',
 'sbi_neg_flag',
 'secured_close_last3mnths',
 'totl_neg_ratio',
 'two_wheeler_close_count_last3mnths',
 'two_wheeler_count',
 'written_off_accounts']
# best_features=['age','auto_loan_count','cc_neg_flag','cfp_interest','city_band_Chennai','city_band_Delhi/NCR','city_band_Hyderabad','city_band_Kolkata','city_band_Missing/Others','city_band_Mumbai','city_band_Pune','cl_neg_flag','closed_accounts','closed_accounts_last3mnths','consumer_loan_count','contactability','credit_card_count','credit_limit','credit_score','email_login_flag_One','email_login_flag_Zero','employment_type_Self Employed','flows_accounts','gold_loan_count','hdfc_neg_flag','housing_loan_count','icici_neg_flag','individual_account','individual_close_account_last3mnths','joint_account','last_login_vintage_M1toM2','last_login_vintage_M3toM6','last_login_vintage_M7toM12','marital_status_Single','max_dpd_on_m3','max_score','monthly_income','neg_acc_on_m12','neg_acc_on_m3','neg_acc_on_m6','neg_acc_on_m9','negative_status_flag_One','negative_status_flag_Three','negative_status_flag_Two','nsaleable','open_accounts','open_negative_status_count','open_total_ratio','personal_loan_count','pl_neg_flag','positive_response_rate','ptp_last_three_months_flag','repayment_tenure','salary_account_AXIS','salary_account_CITI','salary_account_HDFC','salary_account_HDFC Bank','salary_account_I receive by cash','salary_account_ICICI','salary_account_JAMMU & KASHMIR','salary_account_SBI','salary_account_Unknown','salary_account_YESBANK','salary_account_andhra bank','salary_account_otherBank','saleable_accounts_count','sbi_neg_flag','score_on_m12','score_on_m3','score_on_m6','score_on_m9','secured','settled_accounts','settled_flag','sms_login_flag_One','sms_login_flag_Zero','total_amt_sanctioned','total_balance','total_emi_amount','totl_neg_ratio','unique_days_logged_in_flag_One','unique_days_logged_in_flag_Three','unique_days_logged_in_flag_Two','unsecured','unsecured_close_last3mnths','written_off_accounts']
'''green'''
# worst_features=['two_wheeler_count', 'citi_neg_flag', 'auto_loan_close_count_last3mnths', 'gold_loan_close_count_last3mnths', 'consumer_loan_close_count_last3mnths', 'housing_loan_close_count_last3mnths', 'credit_card_close_count_last3mnths', 'two_wheeler_close_count_last3mnths', 'secured_close_last3mnths', 'negative_status_close_count_last3mnths', 'joint_account_close_last3mnths', 'max_dpd_on_m9', 'max_dpd_on_m12', 'max_dpd_on_m6','nsaleable', 'monthly_income', 'age', 'positive_response_rate', 'settled_flag', 'totl_neg_ratio', 'saleable_accounts_count', 'gold_loan_count', 'housing_loan_count', 'credit_card_count', 'open_accounts', 'closed_accounts', 'settled_accounts', 'written_off_accounts', 'open_negative_status_count', 'joint_account', 'total_emi_amount', 'hdfc_neg_flag', 'sbi_neg_flag', 'icici_neg_flag', 'cc_neg_flag', 'pl_neg_flag', 'cl_neg_flag', 'individual_close_account_last3mnths', 'closed_accounts_last3mnths', 'score_on_m9', 'score_on_m12', 'score_on_m6', 'neg_acc_on_m9', 'neg_acc_on_m3', 'neg_acc_on_m12', 'neg_acc_on_m6', 'max_dpd_on_m3', 'salary_account_ACCOUNT', 'salary_account_ACCOUNT TRANSFER', 'salary_account_ALLAHABAD BANK', 'salary_account_ANDHRA', 'salary_account_ANDHRA BANK', 'salary_account_ANDRA BANK', 'salary_account_ANNAI SAI MAGAR BANK', 'salary_account_AU SMALL FINANCE', 'salary_account_AXIS', 'salary_account_Allahabad BAnk', 'salary_account_Allahabad bank', 'salary_account_Andhar Bank', 'salary_account_Andhara bank', 'salary_account_AndhraBank', 'salary_account_Axis', 'salary_account_BANK OF BARODA', 'salary_account_BANK OF INDIA', 'salary_account_BANK OF MAHARASHTRA', 'salary_account_BANK OF MAHARASTRA', 'salary_account_BANK OF MAHARASTRA/BANK OF BARODA', 'salary_account_BANK of BARODA', 'salary_account_BOB', 'salary_account_BOI', 'salary_account_BOM', 'salary_account_BY CASH', 'salary_account_Bandhan bank', 'salary_account_Bank of Baroda', 'salary_account_Bank of India', 'salary_account_Bank of Maharashtra', 'salary_account_Bank of baroda', 'salary_account_Bank of india', 'salary_account_Bank of maharashtra', 'salary_account_Bank or maharashtra', 'salary_account_CANA', 'salary_account_CANARA BANK', 'salary_account_CASH', 'salary_account_CBI', 'salary_account_CBQ', 'salary_account_CENTRAL BANK OF INDIA', 'salary_account_CITI', 'salary_account_CITI UNION BANK', 'salary_account_CITY UNION', 'salary_account_CORPORATIOM', 'salary_account_CORPORATION', 'salary_account_CORPORATION BANK', 'salary_account_COSMO BANK', 'salary_account_COSMOS BANK', 'salary_account_Canara', 'salary_account_Canara Bank', 'salary_account_Canara bank', 'salary_account_Cash', 'salary_account_Catholic Syrian bank', 'salary_account_Central Bank of India', 'salary_account_Central bank of india', 'salary_account_Co-operative Bank', 'salary_account_Corp Bank', 'salary_account_Corporation Bank', 'salary_account_Corporation bank', 'salary_account_DENA', 'salary_account_DEUTSCHE BANK', 'salary_account_DUTCH BANK', 'salary_account_Dena bank', 'salary_account_Deutsche Bank', 'salary_account_Dhanalaxmi Bank', 'salary_account_Direct Credit in union bank of india', 'salary_account_FEDERAL BANK', 'salary_account_FORD', 'salary_account_Federal Bank', 'salary_account_GDCC Bank Pvt. Ltd', 'salary_account_HAND CASH', 'salary_account_HDFC Bank', 'salary_account_HINDUSTHAN', 'salary_account_HSBC', 'salary_account_HSBC Electronic Data Processing Pvt Ltd', 'salary_account_HSBC bank', 'salary_account_Hsbc', 'salary_account_I receive by cash', 'salary_account_I receive by cheque', 'salary_account_I received in cash', 'salary_account_ICICI', 'salary_account_IDBI', 'salary_account_IDBI BANK', 'salary_account_IDBI BANK LTD.', 'salary_account_IDBI Bank', 'salary_account_IDFC', 'salary_account_INDIAN', 'salary_account_INDIAN BANK', 'salary_account_INDIAN OVERSEAS', 'salary_account_INDUSIND', 'salary_account_INDUSIND BANK', 'salary_account_ING', 'salary_account_ING Vysya', 'salary_account_ING is Now Kotak', 'salary_account_IOB', 'salary_account_IOB BANK', 'salary_account_IOB Bank', 'salary_account_Idbi', 'salary_account_Idbi bank', 'salary_account_Indian Bank', 'salary_account_Indian bank', 'salary_account_Indian overseas bank', 'salary_account_Indian oversies bank', 'salary_account_Indianbank', 'salary_account_Indusind', 'salary_account_Indusind Bank', 'salary_account_Indusind bank', 'salary_account_Indusind bank ltd', 'salary_account_Ing vysya bank', 'salary_account_JANATA SAHAKARI BANK', 'salary_account_Janata sahakari bank', 'salary_account_KALYAN JANATA SAHAKARI', 'salary_account_KARNATAKA BANK', 'salary_account_KARUR VYSYA', 'salary_account_KARYR VYSYA BANK', 'salary_account_KMB', 'salary_account_KOTAK', 'salary_account_KOTAK MAHINDAR BANK', 'salary_account_KOTAK MAHINDRA', 'salary_account_KOTAK MAHINDRA BANK', 'salary_account_KOTAK MAHINDRA BANL LTD', 'salary_account_KOTAK MAHINDRA bank', 'salary_account_KVB', 'salary_account_Karnataka bank Ltd', 'salary_account_Karur Vysya Bank', 'salary_account_Kotak', 'salary_account_Kotak Mahendra bank', 'salary_account_Kotak Mahindra', 'salary_account_Kotak Mahindra Bank', 'salary_account_Kotak Mahindra bank', 'salary_account_Kotak bank', 'salary_account_Kotak mahendra bank', 'salary_account_Kotak mahindra', 'salary_account_Kotak mahindra bank', 'salary_account_LVB', 'salary_account_Mahindra kotak', 'salary_account_NRI', 'salary_account_OBC', 'salary_account_ORIENTAL BANK OF COMMERCE', 'salary_account_Oriental Bank of Commerce', 'salary_account_PMC bank', 'salary_account_PNB', 'salary_account_POSTOFFICE      SALARY ACCOUNT', 'salary_account_PUNJAB NATIONAL BANK', 'salary_account_Punjab National Bank', 'salary_account_Punjab national bank', 'salary_account_RBL', 'salary_account_RBL BANK', 'salary_account_RBL Bank', 'salary_account_Ratnakar Bank', 'salary_account_Ratnakar Bank Limited', 'salary_account_SBBJ', 'salary_account_SBH', 'salary_account_SBI', 'salary_account_SBI GROUP', 'salary_account_SBM', 'salary_account_SBP', 'salary_account_SBT', 'salary_account_SCB', 'salary_account_SOUTH INDIA BANK', 'salary_account_SOUTH INDIAN', 'salary_account_SOUTH INDIAN BANK', 'salary_account_STANDARD', 'salary_account_STANDARD CHARTED', 'salary_account_STANDARD CHARTED BANK', 'salary_account_STANDARD CHARTERED', 'salary_account_STANDARD CHARTERED BANK', 'salary_account_STANDRED CHARTRED BANK', 'salary_account_STATE BANK OF BIKANER & JAIPUR', 'salary_account_STATE BANK OF HYDERABAD', 'salary_account_STATE BANK OF MYSORE', 'salary_account_STATEBANK OF TRAVANCORE', 'salary_account_STB', 'salary_account_SYNDICATE', 'salary_account_SYNDICATE BANK', 'salary_account_Saraswat bank', 'salary_account_Sarswath cooperative bank', 'salary_account_Sbbj', 'salary_account_South Indian Bank', 'salary_account_South Indian bank', 'salary_account_Standard Charted Bank', 'salary_account_Standard Charterd Bank', 'salary_account_Standard Chartered', 'salary_account_Standard Chartered Bank', 'salary_account_Standard Chartered bank', 'salary_account_Standard Chattered', 'salary_account_Standard chartered', 'salary_account_Standard chartered bank', 'salary_account_Standart chartered', 'salary_account_Standered charted bank', 'salary_account_State Bank Of Tranvakoor', 'salary_account_State Bank of Hyderabad', 'salary_account_State bank of Hyderabad', 'salary_account_State bank of hyderabad', 'salary_account_State bank of mysore', 'salary_account_Syndicate Bank', 'salary_account_Syndicate bank', 'salary_account_Syndicatebank', 'salary_account_TAMILNAD MERCANTILE BANK', 'salary_account_THE FEDERAL BANK LTD', 'salary_account_THROUGH BANK ACCOUNT', 'salary_account_TMB', 'salary_account_The Ahmedabad District Cooperative bank Ltd', 'salary_account_The Saraswat co-operative Bank', 'salary_account_Transfer through UAE', 'salary_account_UBI', 'salary_account_UCO', 'salary_account_UCO BANK', 'salary_account_UNION', 'salary_account_UNION BANK OF INDIA', 'salary_account_UNITED BANK OF INDIA', 'salary_account_Ubi', 'salary_account_Uco bank', 'salary_account_Union Bank of India', 'salary_account_Union bank', 'salary_account_Union bank of india', 'salary_account_Unknown', 'salary_account_Uttar banga kheyriya gramin bank', 'salary_account_VIJAYA BANK', 'salary_account_Vijaya Bank', 'salary_account_Vijaya bank', 'salary_account_YES', 'salary_account_YES BANK', 'salary_account_YESBANK', 'salary_account_Yes', 'salary_account_Yes Bank', 'salary_account_Yes bank', 'salary_account_andhra bank', 'salary_account_andhrabank', 'salary_account_axis bank', 'salary_account_bank', 'salary_account_bank of baroda', 'salary_account_bank of india', 'salary_account_bank of maharashtra', 'salary_account_bob', 'salary_account_boi', 'salary_account_bom', 'salary_account_by cash', 'salary_account_canara', 'salary_account_canara bank', 'salary_account_cash', 'salary_account_cbi', 'salary_account_central bank of india', 'salary_account_citibank', 'salary_account_city union bank', 'salary_account_co operative bank', 'salary_account_co-operative', 'salary_account_corparation bank', 'salary_account_corporation', 'salary_account_corporation Bank', 'salary_account_corporation bank', 'salary_account_creditted to bank', 'salary_account_deutsche bank', 'salary_account_dhanalakshmi bank', 'salary_account_federal bank', 'salary_account_hdfc', 'salary_account_hsbc', 'salary_account_idbi', 'salary_account_idbi bank', 'salary_account_img', 'salary_account_indian', 'salary_account_indian bank', 'salary_account_indian overseas bank', 'salary_account_indusind', 'salary_account_indusind bank', 'salary_account_indusinda bank', 'salary_account_indusinf', 'salary_account_ing', 'salary_account_ing vysa', 'salary_account_iob', 'salary_account_karur vysya bank', 'salary_account_kotak', 'salary_account_kotak BANK', 'salary_account_kotak Mahindra Bank', 'salary_account_kotak bank', 'salary_account_kotak mahindra', 'salary_account_kotak mahindra bank', 'salary_account_lakshmi vilas bank', 'salary_account_neft', 'salary_account_obc', 'salary_account_oriental bank of commerce', 'salary_account_other', 'salary_account_otherBank', 'salary_account_others', 'salary_account_pnb', 'salary_account_punjab national bank', 'salary_account_receiveByCashOrCheque', 'salary_account_saraswath co-operativa bank', 'salary_account_sbbj', 'salary_account_sbh', 'salary_account_sbi', 'salary_account_sbm', 'salary_account_south Indian Bank', 'salary_account_standard Charatered', 'salary_account_standard charted', 'salary_account_standard charted bank', 'salary_account_standard chartered', 'salary_account_standard chartered bank', 'salary_account_standard chatered', 'salary_account_standerd charted', 'salary_account_state Bank of travancore', 'salary_account_state bank of hyderabad', 'salary_account_state bank of mysore', 'salary_account_state bank of patila', 'salary_account_state bank of travancore', 'salary_account_statebankofhyderabad', 'salary_account_suh', 'salary_account_syndicate', 'salary_account_syndicate bank', 'salary_account_through bank', 'salary_account_ubi', 'salary_account_union bank of india', 'salary_account_yes bank', 'salary_account_yesbank', 'marital_status_Single', 'employment_type_Self Employed', 'city_band_Chennai', 'city_band_Kolkata', 'city_band_Pune', 'email_login_flag_One', 'unique_days_logged_in_flag_Three', 'negative_status_flag_One']
try:
    for feature in worst_features:
        processed_dataset.drop([feature],axis=1,inplace=True)
except:
    pass
# processed_dataset=processed_dataset[best_features]
processed_dataset.head()


,user_id,monthly_income,credit_score,age,ptp_last_three_months_flag,positive_response_rate,contactability,settled_flag,open_total_ratio,cfp_interest,auto_loan_count,gold_loan_count,consumer_loan_count,housing_loan_count,credit_card_count,personal_loan_count,open_accounts,closed_accounts,settled_accounts,flows_accounts,secured,unsecured,individual_account,joint_account,total_amt_sanctioned,total_balance,credit_limit,repayment_tenure,total_emi_amount,personal_loan_close_count_last3mnths,individual_close_account_last3mnths,closed_accounts_last3mnths,unsecured_close_last3mnths,max_score,score_on_m9,score_on_m3,score_on_m12,score_on_m6,salary_account_AXIS,salary_account_HDFC,salary_account_I receive by cash,salary_account_ICICI,salary_account_KOTAK,salary_account_Other,salary_account_SBI,marital_status_Single,employment_type_Self Employed,city_band_Chennai,city_band_Delhi/NCR,city_band_Hyderabad,city_band_Kolkata,city_band_Missing/Others,city_band_Mumbai,city_band_Pune,email_login_flag_One,email_login_flag_Zero,unique_days_logged_in_flag_One,unique_days_logged_in_flag_Three,unique_days_logged_in_flag_Two,sms_login_flag_One,sms_login_flag_Zero,last_login_vintage_M1toM2,last_login_vintage_M3toM6,last_login_vintage_M7toM12,cfp_subscribed
0,2042889,0.23,-0.24,1.65,-0.08,-0.19,-0.20,0.45,-0.14,-0.58,-0.45,-0.48,0.60,1.75,0.96,0.81,0.47,0.54,0.54,0.50,0.60,0.55,0.50,-0.48,0.71,0.71,0.64,0.45,0.53,-0.28,-0.34,-0.34,-0.32,0.38,-0.41,-0.85,-0.36,-0.49,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,26573549,-0.30,-0.69,-1.35,-0.08,-0.19,-0.20,0.45,0.18,-0.58,-0.45,-0.48,-1.12,1.75,0.61,-1.18,-0.12,-0.10,-0.08,-0.03,0.74,-0.61,-0.81,2.68,-0.12,0.31,-0.24,0.12,0.48,-0.28,-0.34,-0.34,-0.32,-0.86,-0.41,-0.85,-0.36,-0.49,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0
2,23202365,0.93,-0.24,-0.31,-0.08,-0.19,-0.20,0.45,-0.37,-0.58,-0.45,2.93,-1.12,-0.41,-1.00,2.10,0.78,2.28,2.28,0.79,2.14,2.74,2.94,-0.48,0.88,0.29,-0.24,0.89,0.85,3.57,2.97,2.93,3.08,-0.14,3.12,-0.85,-0.36,-0.49,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,1,0,0,0
3,21726339,-0.34,-0.48,0.96,-0.08,-0.19,-0.20,0.45,0.14,-0.58,-0.45,-0.48,2.08,-0.41,-1.00,-1.18,0.67,0.35,0.36,0.69,-1.04,0.52,0.43,-0.48,0.32,0.31,-0.24,0.36,0.50,-0.28,-0.34,-0.34,-0.32,0.38,-0.41,-0.85,-0.36,-0.49,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
4,26883702,-0.41,-0.97,2.11,-0.08,-0.19,-0.20,0.45,-0.15,-0.58,-0.45,-0.48,0.81,-0.41,-1.00,-1.18,0.05,0.29,0.30,-0.03,-1.04,0.18,0.07,-0.48,-0.48,-0.13,-0.24,-0.60,0.22,-0.28,2.97,2.93,3.08,-1.30,-0.41,-0.85,-0.36,-0.49,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0


In [45]:
processed_dataset.shape

(303296, 65)

In [35]:
processed_dataset.isnull().sum()

user_id                                       0
nsaleable                                     0
monthly_income                                0
credit_score                                  0
age                                           0
ptp_last_three_months_flag                    0
positive_response_rate                        0
contactability                                0
settled_flag                                  0
open_total_ratio                              0
totl_neg_ratio                                0
saleable_accounts_count                       0
cfp_interest                                  0
auto_loan_count                               0
gold_loan_count                               0
consumer_loan_count                           0
housing_loan_count                            0
credit_card_count                             0
personal_loan_count                           0
two_wheeler_count                             0
open_accounts                           

In [ ]:
processed_dataset.corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

## Train Test Split

In [46]:
'''  Preparing test and train data '''
X = processed_dataset.loc[:,processed_dataset.columns != 'cfp_subscribed']
y = processed_dataset['cfp_subscribed']
train_X_u,test_X_u,train_y,test_y = train_test_split(X,y,random_state=31,test_size=0.2)
train_X = train_X_u.loc[:,train_X_u.columns != 'user_id']
test_X = test_X_u.loc[:,test_X_u.columns != 'user_id']
train_y.head()

95310     0
265149    0
121350    0
12515     0
92277     0
Name: cfp_subscribed, dtype: int64

## Model Building

### Training

##### Initializing Model, Hyper Parameter Tuning and Model Fitting

In [47]:
#SGD
''' Tuning Grid '''
gridParams = {'loss': ['modified_huber'],
                'penalty':['l1','l2','elasticnet'],
                'max_iter':[10000],
                'random_state' : [13,100,1234],
                'warm_start':[False],
                'n_iter_no_change':[5,10,100],
                'early_stopping':[True],
                'learning_rate':['optimal']
            }

'''Model Object'''
model1 = SGDClassifier()


''' Grid Formation '''
grid1 = GridSearchCV(estimator = model1 , param_grid= gridParams, verbose = 2, cv = 5)
# grid1.fit(train_X, train_y)

In [48]:
#RandomForest
''' Tuning Grid '''
gridParams = {'n_estimators': [100,300],
                'max_features':['sqrt','log2',None],
                'random_state' : [13,100],
                'criterion':['gini', 'entropy'],
                 'max_depth':[20,50],
                'min_samples_split':[3],
                'min_samples_leaf':[3]
             }

'''Model Object'''
model2 = RandomForestClassifier(warm_start=False)

''' Grid Formation '''
grid2 = GridSearchCV(estimator = model2 , param_grid= gridParams, verbose = 2, cv = 5)
# grid2.fit(train_X, train_y)

In [ ]:
ensembled = VotingClassifier(estimators=[('sgd', grid1), ('rf', grid2)], voting='soft')
ensembled.fit(train_X, train_y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=5, penalty=l1, random_state=13, warm_start=False; total time=   0.7s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=5, penalty=l1, random_state=13, warm_start=False; total time=   0.7s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=5, penalty=l1, random_state=13, warm_start=False; total time=   0.7s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=5, penalty=l1, random_state=13, warm_start=False; total time=   0.7s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=5, penalty=l1, random_state=13, warm_start=False; total time=   0.7s
[CV] END early_stopping=True, learning_rate=optimal, loss=

[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=10, penalty=l1, random_state=13, warm_start=False; total time=   1.2s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=10, penalty=l1, random_state=13, warm_start=False; total time=   1.2s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=10, penalty=l1, random_state=13, warm_start=False; total time=   1.2s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=10, penalty=l1, random_state=13, warm_start=False; total time=   1.2s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=10, penalty=l1, random_state=100, warm_start=False; total time=   1.2s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=10, pen

[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=100, penalty=l1, random_state=13, warm_start=False; total time=  10.5s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=100, penalty=l1, random_state=13, warm_start=False; total time=  10.4s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=100, penalty=l1, random_state=13, warm_start=False; total time=  10.6s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=100, penalty=l1, random_state=100, warm_start=False; total time=  10.3s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=100, penalty=l1, random_state=100, warm_start=False; total time=  10.5s
[CV] END early_stopping=True, learning_rate=optimal, loss=modified_huber, max_iter=10000, n_iter_no_change=1

[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=13; total time=  20.4s
[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=13; total time=  20.6s
[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=100; total time=  20.3s
[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=100; total time=  20.3s
[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=100; total time=  20.4s
[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=100; total time=  20.1s
[CV] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=3, 

[CV] END criterion=gini, max_depth=20, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=300, random_state=100; total time= 8.2min
[CV] END criterion=gini, max_depth=20, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=300, random_state=100; total time= 7.6min
[CV] END criterion=gini, max_depth=20, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=300, random_state=100; total time= 7.4min
[CV] END criterion=gini, max_depth=50, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=13; total time=  20.7s
[CV] END criterion=gini, max_depth=50, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=13; total time=  20.6s
[CV] END criterion=gini, max_depth=50, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=13; total time=  20.3s
[CV] END criterion=gini, max_depth=50, max_features=sqrt, min_samples_leaf=3, m

[CV] END criterion=gini, max_depth=50, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=300, random_state=13; total time=10.0min
[CV] END criterion=gini, max_depth=50, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=300, random_state=13; total time= 9.4min
[CV] END criterion=gini, max_depth=50, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=300, random_state=13; total time= 8.1min
[CV] END criterion=gini, max_depth=50, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=300, random_state=13; total time= 7.9min
[CV] END criterion=gini, max_depth=50, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=300, random_state=100; total time= 8.5min
[CV] END criterion=gini, max_depth=50, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=300, random_state=100; total time= 8.8min
[CV] END criterion=gini, max_depth=50, max_features=None, min_samples_leaf=3, mi

[CV] END criterion=entropy, max_depth=20, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=13; total time= 2.8min
[CV] END criterion=entropy, max_depth=20, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=100; total time= 2.8min
[CV] END criterion=entropy, max_depth=20, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=100; total time= 2.7min
[CV] END criterion=entropy, max_depth=20, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=100; total time= 2.5min
[CV] END criterion=entropy, max_depth=20, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=100; total time= 2.6min
[CV] END criterion=entropy, max_depth=20, max_features=None, min_samples_leaf=3, min_samples_split=3, n_estimators=100, random_state=100; total time= 2.5min
[CV] END criterion=entropy, max_depth=20, max_features=None

In [ ]:
# ''' Calibration '''
# calibrator = CalibratedClassifierCV(SGD, method="isotonic")

# ''' Training the model '''
# calibrator.fit(train_X, train_y)

##### Pickling Model

In [ ]:
''' Saving model as pickle object '''
# joblib.dump(grid,"clf_model_basic.pkl")
# joblib.dump(best_features,"clf_model_best_features.pkl")
# joblib.dump(grid,"clf_model_basic_Green.pkl")
# joblib.dump(best_features,"clf_model_best_features_Green.pkl")
# joblib.dump(grid,"clf_model_basic_Red1.pkl")
# joblib.dump(best_features,"clf_model_best_features_Red1.pkl")
# joblib.dump(grid1,"clf_SGD_Red.pkl")
# joblib.dump(grid2,"clf_RF_Red.pkl")
# joblib.dump(grid1,"clf_SGD_Green.pkl")
# joblib.dump(grid2,"clf_RF_Green.pkl")
joblib.dump(ensembled,"clf_Ensembled_Green.pkl")
# joblib.dump(grid1,"clf_SGD_Red.pkl")
# joblib.dump(grid2,"clf_RF_Red.pkl")
# joblib.dump(ensembled,"clf_Ensembled_Red.pkl")

In [ ]:
# grid1=joblib.load('clf_SGD_Green.pkl')
# grid2=joblib.load('clf_RF_Green.pkl')
grid=joblib.load('clf_Ensembled_Green.pkl')
# grid1=joblib.load('clf_SGD_Red.pkl')
# grid2=joblib.load('clf_RF_Red.pkl')
# grid=joblib.load('clf_Ensembled_Red.pkl')

In [ ]:
# best_features = ensembled.best_estimator_
# print(best_features)

In [40]:
best_features1 = grid1.best_estimator_
print(best_features1)

SGDClassifier(early_stopping=True, loss='modified_huber', max_iter=10000,
              n_iter_no_change=10, penalty='l1', random_state=1234)


In [41]:
best_features2 = grid2.best_estimator_
print(best_features2)

RandomForestClassifier(criterion='entropy', max_depth=50, max_features=None,
                       min_samples_leaf=3, min_samples_split=3,
                       random_state=13)


In [42]:
feature_importances_dict1={}
for i in range (0,len(list(best_features1.feature_names_in_))):
    feature_importances_dict1[list(best_features1.feature_names_in_)[i]]=list(best_features1.coef_[0])[i]
dict(sorted(feature_importances_dict1.items(), key=lambda item: item[1]))

{'salary_account_AXIS': -38.98676133959612,
 'salary_account_SBI': -35.98396001209155,
 'city_band_Kolkata': -19.03439945355654,
 'closed_accounts': -15.495054887775556,
 'salary_account_KOTAK': -13.628264835515312,
 'city_band_Pune': -12.535078618910479,
 'city_band_Chennai': -8.475462863442855,
 'city_band_Hyderabad': -7.531753422780126,
 'credit_score': -0.14341431653972486,
 'sms_login_flag_Zero': -0.1324193803825224,
 'open_accounts': -0.09631282107983323,
 'sms_login_flag_One': -0.07910157405163677,
 'max_score': -0.05025991116505565,
 'open_total_ratio': -0.04089200002368061,
 'score_on_m3': -0.024194405496863525,
 'email_login_flag_One': -0.011567380968305233,
 'nsaleable': 0.0,
 'monthly_income': 0.0,
 'age': 0.0,
 'settled_flag': 0.0,
 'totl_neg_ratio': 0.0,
 'saleable_accounts_count': 0.0,
 'auto_loan_count': 0.0,
 'gold_loan_count': 0.0,
 'consumer_loan_count': 0.0,
 'housing_loan_count': 0.0,
 'credit_card_count': 0.0,
 'personal_loan_count': 0.0,
 'two_wheeler_count': 0.0

In [43]:
feature_importances_dict2={}
for i in range (0,len(best_features2.feature_importances_)):
    feature_importances_dict2[best_features2.feature_names_in_[i]]=best_features2.feature_importances_[i]
dict(sorted(feature_importances_dict2.items(), key=lambda item: item[1]))

{'nsaleable': 0.0,
 'totl_neg_ratio': 0.0,
 'saleable_accounts_count': 0.0,
 'two_wheeler_count': 0.0,
 'written_off_accounts': 0.0,
 'open_negative_status_count': 0.0,
 'hdfc_neg_flag': 0.0,
 'citi_neg_flag': 0.0,
 'sbi_neg_flag': 0.0,
 'icici_neg_flag': 0.0,
 'cc_neg_flag': 0.0,
 'pl_neg_flag': 0.0,
 'cl_neg_flag': 0.0,
 'auto_loan_close_count_last3mnths': 0.0,
 'gold_loan_close_count_last3mnths': 0.0,
 'consumer_loan_close_count_last3mnths': 0.0,
 'housing_loan_close_count_last3mnths': 0.0,
 'credit_card_close_count_last3mnths': 0.0,
 'two_wheeler_close_count_last3mnths': 0.0,
 'secured_close_last3mnths': 0.0,
 'negative_status_close_count_last3mnths': 0.0,
 'joint_account_close_last3mnths': 0.0,
 'neg_acc_on_m9': 0.0,
 'neg_acc_on_m3': 0.0,
 'neg_acc_on_m12': 0.0,
 'neg_acc_on_m6': 0.0,
 'max_dpd_on_m9': 0.0,
 'max_dpd_on_m3': 0.0,
 'max_dpd_on_m12': 0.0,
 'max_dpd_on_m6': 0.0,
 'salary_account_BOI': 0.0,
 'salary_account_CITI': 0.0,
 'salary_account_I receive by cash or cheque': 0

### Predicting

##### Predicting the test data

In [ ]:
''' Predicting using the best fit parameters '''
# test_fit = best_features.predict_proba(test_X)
# y_pred = pd.DataFrame(test_fit, columns = ['predicted_value'])
processed_dataset=processed_dataset.loc[:,processed_dataset.columns != 'user_id']
test_fit = grid.predict_proba(processed_dataset)
# test_fit1 = grid1.predict_proba(test_X)
# test_fit2 = grid2.predict_proba(test_X)
y_pred=pd.DataFrame(test_fit)
# y_pred1=pd.DataFrame(test_fit1)
# y_pred2=pd.DataFrame(test_fit2)

In [ ]:
# y_pred['predicted_value'].value_counts()
y_pred_1=pd.DataFrame((y_pred1[1]))
y_pred_2=pd.DataFrame((y_pred2[1]))
y_pred_manual=pd.DataFrame(y_pred1[1]+(2*y_pred2[1]))
# y_pred=pd.DataFrame(y_pred1[1]*y_pred2[1])
y_pred.head()

In [ ]:
# predicted_y=y_pred['predicted_value'].apply(lambda x: 1.00 if x >= 1 else 0.00)
predicted_y=y_pred[1].apply(lambda x: 1 if x > 0.01 else 0)

### Metrics

##### Testing Accuracy

In [ ]:
accuracy_score(test_y, predicted_y)

In [ ]:
cm=ConfusionMatrixDisplay(confusion_matrix(test_y, predicted_y))
cm.plot()
plt.show()

In [ ]:
false_positive_rate, true_positive_rate, threshold1 = roc_curve(test_y, predicted_y)
'roc='+str(roc_auc_score(test_y, predicted_y))

In [ ]:
mae = mean_absolute_error(test_y, predicted_y)
mse = mean_squared_error(test_y, predicted_y)
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = r2_score(test_y, predicted_y)

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

In [ ]:
plt.subplots(1, figsize=(10,10))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### Split to Deciles

In [ ]:
# y_pred_deciles.drop(y_pred_deciles.index, inplace=True)
y_pred_deciles = pd.concat([y_pred[1].reset_index(drop = True), test_X_u['user_id'].reset_index(drop = True)], axis = 1)
y_pred_deciles.rename(columns = {1:'predicted_value'}, inplace = True)
y_pred_deciles.head()

In [ ]:
y_pred_deciles_manual = pd.concat([y_pred_manual[1].reset_index(drop = True), test_X_u['user_id'].reset_index(drop = True)], axis = 1)
y_pred_deciles_manual.rename(columns = {1:'predicted_value'}, inplace = True)
y_pred_deciles_manual.head()

In [ ]:
y_pred_deciles_1 = pd.concat([y_pred_1[1].reset_index(drop = True), test_X_u['user_id'].reset_index(drop = True)], axis = 1)
y_pred_deciles_1.rename(columns = {1:'predicted_value'}, inplace = True)
y_pred_deciles_1.head()

In [ ]:
y_pred_deciles_2 = pd.concat([y_pred_2[1].reset_index(drop = True), test_X_u['user_id'].reset_index(drop = True)], axis = 1)
y_pred_deciles_2.rename(columns = {1:'predicted_value'}, inplace = True)
y_pred_deciles_2.head()

In [ ]:
y_pred_deciles = pd.merge(y_pred_deciles,cfp_backup, on = 'user_id', how = 'left')
y_pred_deciles.head()

In [ ]:
y_pred_deciles_manual = pd.merge(y_pred_deciles_manual,cfp_backup, on = 'user_id', how = 'left')
y_pred_deciles_manual.head()

In [ ]:
y_pred_deciles_1 = pd.merge(y_pred_deciles_1,cfp_backup, on = 'user_id', how = 'left')
y_pred_deciles_1.head()

In [ ]:
y_pred_deciles_2 = pd.merge(y_pred_deciles_2,cfp_backup, on = 'user_id', how = 'left')
y_pred_deciles_2.head()

In [ ]:
y_pred_deciles['decile'] = pd.qcut(y_pred_deciles['predicted_value'].rank(method='first'), 10, labels=False)
y_pred_deciles['decile'] = y_pred_deciles['decile'].apply(lambda x: int(10 - x))
y_pred_deciles['decile'].value_counts()

In [ ]:
y_pred_deciles_manual['decile'] = pd.qcut(y_pred_deciles_manual['predicted_value'].rank(method='first'), 10, labels=False)
y_pred_deciles_manual['decile'] = y_pred_deciles_manual['decile'].apply(lambda x: int(10 - x))
y_pred_deciles_manual['decile'].value_counts()

In [ ]:
y_pred_deciles_1['decile'] = pd.qcut(y_pred_deciles_1['predicted_value'].rank(method='first'), 10, labels=False)
y_pred_deciles_1['decile'] = y_pred_deciles_1['decile'].apply(lambda x: int(10 - x))
y_pred_deciles_1['decile'].value_counts()

In [ ]:
y_pred_deciles_2['decile'] = pd.qcut(y_pred_deciles_2['predicted_value'].rank(method='first'), 10, labels=False)
y_pred_deciles_2['decile'] = y_pred_deciles_2['decile'].apply(lambda x: int(10 - x))
y_pred_deciles_2['decile'].value_counts()

#### Export to CSV

In [ ]:
# y_pred_deciles.to_csv('CFP_deciled_data_Green.csv')
y_pred_deciles.drop(['pincode'],axis=1,inplace=True)
y_pred_deciles_manual.drop(['pincode'],axis=1,inplace=True)
y_pred_deciles_1.drop(['pincode'],axis=1,inplace=True)
y_pred_deciles_2.drop(['pincode'],axis=1,inplace=True)
# y_pred_deciles.to_csv('CFP_deciled_data_Red.csv')
# y_pred_deciles.to_csv('CFP_deciled_data_Green.csv')
# y_pred_deciles_manual.to_csv('CFP_deciled_data_Green_manual.csv')
# y_pred_deciles_1.to_csv('CFP_deciled_data_Green_SGB.csv')
# y_pred_deciles_2.to_csv('CFP_deciled_data_Green_RF.csv')
y_pred_deciles.to_csv('CFP_deciled_data_Red.csv')
y_pred_deciles_manual.to_csv('CFP_deciled_data_Red_manual.csv')
y_pred_deciles_1.to_csv('CFP_deciled_data_Red_SGB.csv')
y_pred_deciles_2.to_csv('CFP_deciled_data_Red_RF.csv')

##### -- The End